# Averages

I used to have all this written down, mostly for myself, on subsurfwiki.org, but I don't maintain that site any more, so here it is, less complete and probably less well done, ha.

**Note.** I learned about the expression of means as "some loss" minimization from the statistician **Tommy Odland** at Equinor's developer conference in 2024. His talks are reliably awesome, you should check them out. [Here's the one about averages.](https://tommyodland.com/articles/2024/the-average-value/)

**More!** For an extensive list of averages, see the Wikipedia article on [Average](https://en.wikipedia.org/wiki/Average).

---

There are a lot of ways to measure the 'centre' of a distribution, some linear, some nonlinear.

We'll get at them with pure Python and/or with NumPy or SciPy. But nothing too fancy.

In [75]:
import numpy as np

x = np.array([-3, -1, 0, 0, 1, 1, 1, 2, 2, 2, 2, 2, 3, 5, 100])

---

## Pythagorean means

We stsart with the famous [Pythagorean means](https://en.wikipedia.org/wiki/Pythagorean_means), for which A &ge; G &ge; H. 

### Arithmetic mean, aka 'the average' or 'the mean'

$$ \mu = \frac{1}{N} \sum_{i=0}^{N} x_i $$

In [71]:
μ = sum(x) / len(x)
μ

np.float64(7.8)

In [72]:
np.mean(x)

np.float64(7.8)

This function is also available as a method on an array:

In [73]:
x.mean()

np.float64(7.8)

We can also compute the mean of the positive values only:

In [74]:
np.mean(x[x > 0])

np.float64(11.0)

### The mean minimizes the L2 norm

Again, this insight is from Tommy Odland, thanks to him.

We can arrive at several of these means via a minimization problem. In the case of the arithmetic mean, we can minimize the Euclidean distance (aka L2 norm, or 'crow flies' distance) from the data to some candidate mean. The mimimum is the arithmetic mean.

(Later on we will see that choosing other distances or norms results in other means.)

I'll use `scipy.optimize.minimize` to solve the problem, but we could use gradient descent or some other method.

In [120]:
from scipy.optimize import minimize

L2 = lambda μ: np.linalg.norm(x - μ, ord=2)

rng = np.random.default_rng(42)
μ0 = rng.random()  # Initial guess is random.

minimize(L2, μ0).x

array([7.80000438])

### Geometric mean

$$  \sqrt[n]{a_1 a_2 \cdots a_n \vphantom{t}} $$

Strictly undefined for datasets containing negative numbers or zeros.

In [99]:
from functools import reduce
from operator import mul

reduce(mul, x)**(1/len(x))

np.float64(0.0)

Let's remove the non-positive numbers.

In [100]:
x_ = x[x > 0]
reduce(mul, x_)**(1/len(x_))

np.float64(2.664203443021162)

And with `scipy.stats`:

In [78]:
from scipy.stats import gmean

gmean(x)

np.float64(nan)

Again, we can compute it for the positive values though:

In [79]:
gmean(x_)

np.float64(2.664203443021162)

### The geometric average minimizes multiplicative loss

In [131]:
def mult_loss(μ):
    return sum((np.log(x_) - np.log(μ))**2)
    
μ0 = rng.random()

minimize(mult_loss, μ0)

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 16.84474243908292
        x: [ 2.664e+00]
      nit: 8
      jac: [-4.768e-07]
 hess_inv: [[ 3.223e-01]]
     nfev: 18
     njev: 9

### Harmonic mean

$$ \textrm{HM} \left( x_1,\; \ldots,\; x_n \right) = \frac{n}{ \frac{1}{x_1} + \;\cdots\; + \frac{1}{x_n}}  $$

Like the geometric mean, it is undefined for datasets containing negative numbers or zeros.

In [97]:
len(x) / sum(1/xi for xi in x)

/tmp/ipykernel_2793850/1809065445.py:1: RuntimeWarning: divide by zero encountered in scalar divide
  len(x) / sum(1/xi for xi in x)


np.float64(0.0)

Let's use the positive-only version of `x`:

In [102]:
len(x_) / sum(1/xi for xi in x_)

np.float64(1.8201875344732488)

And with `scipy`:

In [103]:
hmean(x_)

np.float64(1.820187534473249)

### The harmonic average minimizes relative loss

In [145]:
def relative_loss(μ):
    return sum((1 / x_) * (x_ - μ)**2)
    
μ0 = rng.random()

minimize(relative_loss, μ0)

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 100.9779371207944
        x: [ 1.820e+00]
      nit: 2
      jac: [-9.537e-07]
 hess_inv: [[ 8.274e-02]]
     nfev: 6
     njev: 3

---

## Contraharmonic mean

In [149]:
sum(xi**2 for xi in x) / sum(x)

np.float64(86.04273504273505)

Expressed as an optimization problem:

In [148]:
def loss(μ):
    return sum(x * (x - μ)**2)

μ0 = rng.random()

minimize(loss, μ0)

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 133974.78632478724
        x: [ 8.604e+01]
      nit: 3
      jac: [ 0.000e+00]
 hess_inv: [[ 4.274e-03]]
     nfev: 12
     njev: 6

---

## Median

The median is the central number in a sorted list, or the mean of the 2 central numbers.

In [17]:
np.median(x)

np.float64(2.0)

It may or may not be different for the positive-only array; in our case it is the same.

In [105]:
np.median(x_)

np.float64(2.0)

### The median minimizes the L1 norm

Remember, this algorithm is not guaranteed to converge on a given problem, especially with its default parameters.

In [113]:
L1 = lambda μ: np.linalg.norm(x - μ, ord=1)

μ0 = rng.random()  # Initial guess is random.

minimize(L1, μ0)

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 117.00000001264128
        x: [ 2.000e+00]
      nit: 2
      jac: [ 5.166e-01]
 hess_inv: [[ 5.683e-01]]
     nfev: 204
     njev: 96

---

## Mode

The value or values with the highest count(s). If non-unique (i.e. the dataset is bi-modal or multi-modal), note that `scipy.stats.mode()` reports the minimum of the modes.

Note also, from Wikipedia:

> If X is a discrete random variable, the mode is the value x at which the probability mass function takes its maximum value. In other words, it is the value that is most likely to be sampled [sometimes called "most likely"].

In [57]:
from scipy.stats import mode

mode(x)

ModeResult(mode=np.int64(2), count=np.int64(5))

### The mode minimizes the L0 norm

The L0 norm of an array _a_ is the number of non-zero elements, or `np.count_nonzero(a)`. I need suggestions for a nice way to minimize this for integers. 

---

## Quadratic mean, or RMS

This is useful for understanding the magnitude of a quantity that can vary above and below zero, when the mean may be close to zero (eg when looking at band-limited data such as audio or seismic). Taking the root casts the value back to the original units.

In [121]:
import math

math.sqrt(sum(xi**2 for xi in x) / len(x))

25.90624120426067

In [123]:
np.sqrt(np.mean(x**2))

np.float64(25.90624120426067)

There is also [`sklearn.metrics.root_mean_squared_error()`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.root_mean_squared_error.html).

In [150]:
def loss(μ):
    return sum((x**2 - μ**2)**2)

μ0 = rng.random()

minimize(loss, μ0)

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 93244571.73333335
        x: [ 2.591e+01]
      nit: 3
      jac: [ 0.000e+00]
 hess_inv: [[ 1.293e-05]]
     nfev: 24
     njev: 12

---

## Trimmed mean

We can lose some proportion of the outermost values in the distribution. For example, in the `scipy` implementation, `0.1` means to chop 10% of the values from _each_ end (i.e. 1.5 out of 15 values, which it will round down to 1; to chop 2 values we need a proportion over about 0.14 because 0.14 * 15 = 2.1).

In [96]:
from scipy.stats import trim_mean

trim_mean(x, 0.1)

np.float64(1.5384615384615385)

---

&copy; Matt Hall 2024 / Licensed CC BY, please re-use this work